# Ablation Teacher-Student Subject

In [ ]:
# Imports
# !pip install torchinfo
import torch
from torchinfo import summary
import matplotlib.pyplot as plt
import os
import gc
# os.environ['OPENBLAS_NUM_THREADS'] = '1'
import numpy as np
import TrainerVTS_V08F3_ablation_nots as TP
import DataSetting_v5 as DS

In [ ]:
date = '20240925_A208'
name = f'{date}_Abts_sub'
data_path = [
    '../dataset/Door_EXP/A208',
#             '../dataset/Door_EXP/A308',
#             '../dataset/Door_EXP/B211',
#             '../dataset/Door_EXP/C605'
            ]
level = 'subject'

data_organizer = DS.DataOrganizer(name, data_path, level)
data_organizer.load()

In [ ]:
preprocess = DS.Preprocess(new_size=(128, 128))
data_organizer.load_plan('../dataset/Door_EXP/subject_r0.2_A208.pkl')

def run_t(gpu):
    for i in range(6):
        data_organizer.gen_plan(subset_ratio=0.2)
        train_loader, valid_loader, test_loader, current_test = data_organizer.gen_loaders(mode='s')

        trainer = TP.AbTrainer(beta=0.25,
                                  name='Ab_noTS',
                                  loss_optimizer = {'LOSS': [torch.optim.Adam, 1e-4]},
                                  lr=1e-4, epochs=10, cuda=gpu,
                                      preprocess = preprocess,
                                  train_loader=train_loader, valid_loader=valid_loader, test_loader=test_loader,
                                  notion=f'{name}_{current_test}')
        trained = trainer.schedule(lr_decay=True)
        
        del trainer
        del trained
        del train_loader
        del valid_loader
        del test_loader
        torch.cuda.empty_cache()
        gc.collect()
        break

In [ ]:
gpu = 0
torch.cuda.set_device(gpu)
run_t(gpu)